# Inizializzazione università e SCA

In [1]:
from SmartContractAuthority.sca import SmartContractAuthority
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import serialization, hashes
from University.university import University
from Student.student import Student
import base64
import sys

SCA_PATH = './smart_contract_authority/'
UNIVERSITIES_PATH = './universities/'
STUDENTS_PATH = './students/'
CONTRACTS_BUILD_PATH = './SmartContracts/build'

## Creazione Smart Contract Authority

In [2]:
#SmartContractAuthority.create_sca(SCA_PATH, CONTRACTS_BUILD_PATH)
sca = SmartContractAuthority(SCA_PATH+"SCA", CONTRACTS_BUILD_PATH)

## Creazione Università

In [3]:
#University.create_university(UNIVERSITIES_PATH, CONTRACTS_BUILD_PATH)
uni = University(UNIVERSITIES_PATH+'University_Università Degli Studi Di Salerno', CONTRACTS_BUILD_PATH)
#University.create_university(UNIVERSITIES_PATH, CONTRACTS_BUILD_PATH)
uni2= University(UNIVERSITIES_PATH+'University_UNICAEN', CONTRACTS_BUILD_PATH)

## Creazione studente

In [9]:
#Student.create_student(STUDENTS_PATH)
studente = Student(STUDENTS_PATH+'Student_AntonioCarbone')

Studente 'Antonio Carbone' caricato da ./students/Student_AntonioCarbone


## Accreditamento università create

In [ ]:
UID, sca_contract_address = sca.register_university(uni)
uni.update_uid(UID)
uni.update_sca_contract_address(sca_contract_address)

UID, sca_contract_address = sca.register_university(uni2)
uni2.update_uid(UID)
uni2.update_sca_contract_address(sca_contract_address)

## Immatricolazione presso università di origine

In [ ]:
credenziale_studente,sid = uni.register_student(studente)
studente.set_sid(sid)
studente.save_credential(credenziale_studente)

## Immatricolazione presso università estera

In [ ]:
uni2.register_erasmus_student(studente, studente.credentials[0].toJSON())
uni2.update_erasmus_students(studente, credenziale_studente.CID)

## Richiesta e rilascio credenziale carriera

In [6]:
career_credential = uni2.request_career_credential(studente)
studente.save_credential(career_credential)

1
<class 'int'>
1
Il SID fornito è valido.
Challenge di autenticazione superata.
Registrazione CID completata con successo. Transaction Hash: f99a6d246ec23abb1a2903ae98cca649a4b87580a3cf56a4848ee2a5b914e825
Complimenti Antonio Carbone 
Richiesta della tua credenziale carriera completata!
Arrivederci da UNICAEN!
Credenziale salvata in: ./students/Student_AntonioCarbone\persistency\credentials\credential_1.json


## Costruzione della credenziale parziale

In [7]:
partial_credential = studente.build_shared_credential()

studente.save_credential(partial_credential)


Seleziona una credenziale da condividere:
0: CID = CID:UNISA:1
1: CID = CID:ENSICAEN:5

Seleziona le proprietà da condividere (y/n):

Credenziale parziale costruita con successo:
{
    "certificateId": "CID:ENSICAEN:5",
    "studentId": "1",
    "universityId": "UID:ENSICAEN",
    "issuanceDate": "2023-10-01",
    "properties": [
        {
            "typology": "ErasmusInfo",
            "data": {
                "programName": "Erasmus+ Europe",
                "startActivity": "2024-09-01",
                "endActivity": "2025-01-31"
            },
            "nonce": "17529228065973187",
            "merkle_proof": [
                [
                    "a8b83db15377899d2a1b2a95144da100ddd18d1cf55ad207e5933caf64f9f471",
                    "R"
                ],
                [
                    "9ef87092932fc1210106bc584ac2c4afdfca6fda7fbdeccfdb7dee82cfe8f70e",
                    "R"
                ],
                [
                    "a6bbbd617bbcfd969cd1de9695b0d8e

## Condivisione credenziale

In [12]:
uni.validate_shared_credential(studente, studente.credentials[0].toJSON())

L'università che ha rilasciato la credenziale condivisa è fidata.


Exception: Errore nella verifica del SID contenuto nella credenziale: invalid literal for int() with base 10: 'SID:UNISA:1'

## Revoca credenziale di immatricolazione

In [11]:
uni.revoke_cid("CID:UNISA:1")

Modifica CID completata con successo. Transaction Hash: b3548a892447c3013cdcb511f3d90a2c13ff37f226cd0d3a13e4b8aee2b646d0
Credenziale CID:UNISA:1 revocata con successo!


## Revoca credenziale carriera

In [ ]:
uni2.revoke_cid("CID:ENSICAEN:0")